# Distribución del repositorio
-app

-data
-notebooks



# Data Wrangling

Comencé este examen siguiendo los pasos para la lectura y pre procesado inicial del dataset, tomé el Json, lo normalicé, recorté el dataset a las variables de importancia, las renombré, hice el feature engineering para la variable de respuesta y comencé a ir i venir al EDA para obtener insights

Primero hice limṕieza clásica de textos, eliminé todo lo que no fuera alfanumérico, estandaricé a minúsculas, expandí contracciones y quité stopwords

Después cuando comencé a checar el EDA noté que había palabras que eran casi constantes a lo largo de todas las quejas, así que decidí generar una función que recortara aquellas palabras que aparecieran en cierta cantidad de documentos y finalmenté la dejé para el 80% de los documentos.

Casi para terminar noté que había categorías con muy pocos registros, así que tomé un approach similar al de las palabras muy repetidas y eliminé todas aquellas categorías con un conteo de registros menor a 10

Por último apliqué ambos métodos en batch processing y guardé un save state del dataset en transformed data

# EDA

El EDA fue mi guía para el data wranglin, ya que los insights que iba obteniendo e servían para la limpieza, entre lo más relevante está el notar las clases con pocos registros, mucho de stopwords y palabras muy repetidas, fue un procceso iterativo pero finalmente dejé limpio el dataset y procedí a la visualización

Para ello reutilicé una función que ya había hecho hace tiempo para graficar un wordcloud por cada categoría junto a una colección del top 10 unigramas, bigramas y trigramas, para evalúar que tan distintas eran las categorías después de la limpieza, y los resultados fueron satisfactorios
